# Experimentos: Baseline com QSVM

Este notebook estabelece baselines de performance com QSVM no dataset Student Performance

Usando Seleção de features com base 


In [1]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import time

from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import StatevectorSampler
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC

# Dataset = Student Performance

In [ ]:
# Importando o dataset
student_performance = '../data/student-mat.csv'

df_student_performance = pd.read_csv(student_performance, sep=',')  # Lendo o dataset com separador ','

# y são as colunas de notas
y = df_student_performance[['G1', 'G2', 'G3']]
# X são todas as outras colunas
X = df_student_performance.drop(columns=['G1', 'G2', 'G3'])


print("--- Dataset carregado localmente com sucesso! ---")
print("Formato das features (X):", X.shape)
print("Formato dos alvos (y):", y.shape)

print("\n5 primeiras linhas das features (X):")


FileNotFoundError: [Errno 2] No such file or directory: 'data/student-mat.csv'

In [ ]:
X.head(5) 

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,no,4,3,4,1,1,3,4
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,no,5,3,3,1,1,3,2
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,no,4,3,2,2,3,3,6
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,3,2,2,1,1,5,0
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,no,no,4,3,2,1,2,5,0


In [ ]:
# Dimensões dos dados do Student Performance Dataset
print("\nDimensões do Student Performance Dataset:")
print(f"X: {X.shape}")
print(f"y: {y.shape}")



Dimensões do Student Performance Dataset:
X: (649, 30)
y: (649, 3)


In [ ]:
# Data cleaning
# check for missing values
print("\nVerificando valores ausentes:")
print(X.isnull().sum())


Verificando valores ausentes:
school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
dtype: int64


## Pré-processamento dataset Student Performance

In [ ]:
# creating classification targets
y_final = np.where(y['G3'] >= 10, 1, 0)  # Convertendo a nota final em uma classificação binária (aprovado/reprovado). Usando 10 como o limite de aprovação.
# 1 representa aprovado e 0 representa reprovado.
# Verificando a distribuição das classes
print("\nDistribuição das classes (Aprovado/Reprovado):")
print(pd.Series(y_final).value_counts()) # Contando as ocorrências de cada classe. usando pd.Series para criar uma série pandas a partir do array numpy





Distribuição das classes (Aprovado/Reprovado):
1    549
0    100
Name: count, dtype: int64


In [ ]:
# The dataset seem unbalanced, so it has to be balanced

# tranforming text features into numerical features
X_final = pd.get_dummies(X, drop_first=True)  # Convertendo variáveis categóricas em variáveis dummy # drop_first=True evita a armadilha da variável fictícia
# Verificando as dimensões dos dados após a codificação
print("\nDimensões dos dados após a codificação:") 
print("Formato das features (X) antes do processamento:", X.shape)
print("Formato das features (X) após o processamento:", X_final.shape)
# Verificando as primeiras linhas dos dados codificados
print("\nPrimeiras linhas dos dados codificados:")
print(X_final.head())
print(X_final.dtypes)




Dimensões dos dados após a codificação:
Formato das features (X) antes do processamento: (649, 30)
Formato das features (X) após o processamento: (649, 39)

Primeiras linhas dos dados codificados:
   age  Medu  Fedu  traveltime  studytime  failures  famrel  freetime  goout  \
0   18     4     4           2          2         0       4         3      4   
1   17     1     1           1          2         0       5         3      3   
2   15     1     1           1          2         0       4         3      2   
3   15     4     2           1          3         0       3         2      2   
4   16     3     3           1          2         0       4         3      2   

   Dalc  ...  guardian_mother  guardian_other  schoolsup_yes  famsup_yes  \
0     1  ...             True           False           True       False   
1     1  ...            False           False          False        True   
2     2  ...             True           False           True       False   
3     1  ...     

## Spliting data into training and testing sets

In [ ]:
X_train_full, X_test_full, y_train, y_test = train_test_split(
    X_final, y_final, test_size=0.25, random_state=42, stratify=y_final
)



## Applying feature selection by correlation (30 -> 5 features)

In [ ]:
train_df = X_train_full.copy()
train_df["aprovado"] = y_train

# Calculamos a correlação das features com a variável alvo
correlation = train_df.corr(numeric_only=True)["aprovado"].abs().sort_values(ascending=False)
# Selecionando as 5 features mais importantes (excluindo a própria 'aprovado')
N_FEATURES = 5
top_features = correlation[1:N_FEATURES+1].index.tolist()
print(f"\nTop {N_FEATURES} features selecionadas com base no treino:", top_features)


X_train_selected = X_train_full[top_features]
X_test_selected = X_test_full[top_features]


Top 5 features selecionadas com base no treino: ['failures', 'higher_yes', 'school_MS', 'Medu', 'studytime']


## Normalizing 

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)
print("\nDados selecionados e normalizados.")


Dados selecionados e normalizados.


## Using SMOTE to balance

In [ ]:
# Using SMOTE to balance the dataset
from imblearn.over_sampling import SMOTE

print("Distribuição de classes no treino ANTES do SMOTE:")
print(pd.Series(y_train).value_counts())

# aplying SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Verificando o balanceamento DEPOIS do SMOTE
print("\nDistribuição de classes no treino DEPOIS do SMOTE:")
print(pd.Series(y_train_resampled).value_counts())

Distribuição de classes no treino ANTES do SMOTE:
1    411
0     75
Name: count, dtype: int64

Distribuição de classes no treino DEPOIS do SMOTE:
1    411
0    411
Name: count, dtype: int64


## Training the dataset

In [ ]:
# Treinamento e Avaliação do QSVM
num_features = X_train_resampled.shape[1]
feature_map_sp = ZZFeatureMap(feature_dimension=num_features, reps=1)
fidelity_kernel_sp = FidelityQuantumKernel(feature_map=feature_map_sp)
qsvc_sp = QSVC(quantum_kernel=fidelity_kernel_sp, random_state=42)

print(f"\nModelo QSVC criado para {num_features} features. Iniciando treinamento...")

start_time = time.time()
qsvc_sp.fit(X_train_resampled, y_train_resampled)
end_time = time.time()
print(f"Modelo treinado em {end_time - start_time:.2f} segundos.")

qsvc_predictions_sp = qsvc_sp.predict(X_test_scaled)

print("\n--- Relatório de Classificação para o QSVM com Seleção de Features (Student Performance) ---")
print(classification_report(y_test, qsvc_predictions_sp, target_names=['Reprovado', 'Aprovado']))


Modelo QSVC criado para 5 features. Iniciando treinamento...
Modelo treinado em 858.06 segundos.

--- Relatório de Classificação para o QSVM com Seleção de Features (Student Performance) ---
              precision    recall  f1-score   support

   Reprovado       0.31      0.56      0.40        25
    Aprovado       0.91      0.78      0.84       138

    accuracy                           0.74       163
   macro avg       0.61      0.67      0.62       163
weighted avg       0.82      0.74      0.77       163

